In [2]:
!pip install -q datasets transformers qwen_vl_utils


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [15]:
!pip install -q accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import io
import json
from PIL import Image
from datasets import load_dataset
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, Qwen2VLProcessor
#from trl import SFTConfig, SFTTrainer
#from peft import LoraConfig
from qwen_vl_utils import process_vision_info



In [2]:
# 시스템(assistant)에게 주어진 역할
system_message = "당신은 이미지로부터 문자와 숫자를 판독하는 전문적인 OCR 모델입니다"

# 실제로 사용자 입력 -> 모델이 답해야 하는 프롬프트
prompt = """입력 정보:
- image: [image]

위 이미지에서 모든 문자,숫자를 판독해서 텍스트로 결과를 출력하세요.
출력 시 **아래 TEXT 형태**를 반드시 지키세요:
result
원재료명 설탕,물엿,젤라틴,D-소르비톨(감미료),산도조절제I,산도조절제II,산도조절제III,혼합제제I(젓산,젓산칼슘,이산화규소),향료3종,당류가공품,펙틴,산도조절제IV,블랙캐롯농축액,혼합재제II(홍화황색소,치자청색소,주정),천연향료(콜라향)
복숭항,밀,우유,대두,땅콩,달걀,토마토 혼입 가능
복숭항,밀,우유,대두,땅콩,달걀,토마토 와 같은 제조시설에서 제조
"""

In [3]:
import io
from PIL import Image
from IPython.display import display
import base64

def load_and_display_image(file_path):
  """
  로컬 JPG 파일을 불러와서 코랩 출력창에 표시합니다.

  Args:
    file_path (str): 읽어올 JPG 파일의 경로.
  """
  try:
    with open(file_path, "rb") as f:
      image_data = f.read()

    # 이미지 데이터를 메모리 버퍼에 씁니다.
    buffer = io.BytesIO(image_data)

    # 버퍼의 시작점으로 돌아갑니다.
    buffer.seek(0)

    # 버퍼에서 이미지를 엽니다.
    image = Image.open(buffer)

      # 이미지를 Base64 문자열로 인코딩
    buffer_for_b64 = io.BytesIO()
    image.save(buffer_for_b64, format='PNG')
    encoded_string = base64.b64encode(buffer_for_b64.getvalue()).decode('utf-8')

    # IPython.display.display 함수를 사용하여 이미지를 출력합니다.
    #print("이미지를 출력합니다=",len(encoded_string))
    #display(image)

    return image, encoded_string

  except FileNotFoundError:
    print(f"Error: 파일 '{file_path}'을(를) 찾을 수 없습니다.")
  except Exception as e:
    print(f"Error: 이미지를 불러오는 중 오류가 발생했습니다: {e}")



In [4]:

def format_data(image_path, txt_path):
   #image_path = '/content/김광무_100.jpg'
   #image, encoded_string = load_and_display_image(image_path)

   return {
       "messages": [
           {
               "role": "system",
               "content": [
                   {
                       "type": "text",
                       "text": system_message
                   }
               ],
           },
           {
               "role": "user",
               "content": [
                   {
                       "type": "image",
                       "image": image_path,
                   },
               ],
           },
           {
               "role": "assistant",
               "content": [
                   {
                       "type": "text",
                       "text": "",
                   }
               ],
           },
       ],
      "ocr_file_name": txt_path,
   }

In [5]:
import os

folder_path = "./data"
train_list= []

for file_name in os.listdir(folder_path):
  
  if file_name.lower().endswith(".jpg"):
    #print("file_name=",file_name)
    # 파일의 전체 경로를 만듭니다.
    full_path = os.path.join(folder_path, file_name)
    
    # 1. 파일명에서 확장자를 제외한 부분(stem)을 가져옵니다.
    file_stem, _ = os.path.splitext(file_name)
    
    # 2. 새로운 파일명(original_name_ocr.txt)을 만듭니다.
    new_file_name = f"{file_stem}_ocr.txt"
    
    # 3. 원래 파일의 폴더 경로와 새로운 파일명을 합쳐서 새 경로를 만듭니다.
    txt_path = os.path.join(folder_path, new_file_name)
    
    train_list.append(format_data(full_path, txt_path))
    


In [6]:
print(len(train_list), train_list[2])

62 {'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 이미지로부터 문자와 숫자를 판독하는 전문적인 OCR 모델입니다'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': './data/김광무_103.jpg'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': ''}]}], 'ocr_file_name': './data/김광무_103_ocr.txt'}


In [7]:
model_id = "Qwen/Qwen2-VL-7B-Instruct"

model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)

processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [8]:
sample_data = train_list[0]['messages']

sample_prompt = processor.apply_chat_template(
    sample_data,
    tokenize=False,
    add_generation_prompt=False,
)

In [9]:
def split_input_and_label(prompt):
    input = prompt.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = prompt.split('<|im_start|>assistant')[1]
    return input, label

In [10]:
sample_prompt_input, sample_prompt_label = split_input_and_label(sample_prompt)

In [11]:
sample_data_image_inputs, sample_data_video_inputs = process_vision_info(sample_data)

In [12]:
inputs = processor(
    text=[sample_prompt_input],
    images=sample_data_image_inputs,
    videos=sample_data_video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [13]:
%%time
# Inference: Generation of the output
generated_ids = model.generate(
    **inputs,
    max_new_tokens=2048,
    temperature=0.1,
    top_p=0.001,
    repetition_penalty=1.05,
)

generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print("추론결과=\n")
print(output_text)


추론결과=

['\n\n이 이미지에는 다음과 같은 정보가 포함되어 있습니다:\n\n- 제품 이름: SPAM\n- 제품 번호: 20080405015-227\n- 포장재질: 폴리에틸렌\n- 보관 방법: 0~10℃ 냉장보관\n- 총 내용량: 605g\n- 100g 당 칼로리: 285kcal\n- 나트륨: 930mg (47%)\n- 탄수화물: 8g (2%)\n- 당류: 1g (미만)\n- 지방: 23g (43%)\n- 트랜스지방: 0g\n- 포화지방: 7g (47%)\n- 콜레스테롤: 50mg (17%)\n- 단백질: 12g (22%)\n\n또한, 이 제품은 소비기한이 있으며, 별도의 기일까지 소비가 가능한 것으로 보입니다.']
CPU times: user 12.2 s, sys: 552 ms, total: 12.8 s
Wall time: 11.5 s


In [18]:
print(output_text[0])



이 이미지에는 다음과 같은 정보가 포함되어 있습니다:

- 제품 이름: SPAM
- 제품 번호: 20080405015-227
- 포장재질: 폴리에틸렌
- 보관 방법: 0~10℃ 냉장보관
- 총 내용량: 605g
- 100g 당 칼로리: 285kcal
- 나트륨: 930mg (47%)
- 탄수화물: 8g (2%)
- 당류: 1g (미만)
- 지방: 23g (43%)
- 트랜스지방: 0g
- 포화지방: 7g (47%)
- 콜레스테롤: 50mg (17%)
- 단백질: 12g (22%)

또한, 이 제품은 소비기한이 있으며, 별도의 기일까지 소비가 가능한 것으로 보입니다.


In [19]:
%%time
from tqdm import tqdm

for index, item in enumerate(tqdm(train_list)):
    test_data = item['messages']
    txt_path = item['ocr_file_name']
    print("처리중 txt_path=", txt_path)

    test_prompt = processor.apply_chat_template(
        test_data,
        tokenize=False,
        add_generation_prompt=False,
    )
    
    test_prompt_input, test_prompt_label = split_input_and_label(test_prompt)
    test_data_image_inputs, test_data_video_inputs = process_vision_info(test_data)
    
    inputs = processor(
        text=[test_prompt_input],
        images=test_data_image_inputs,
        videos=test_data_video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")
    
    # Inference: Generation of the output
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=2048,
        temperature=0.1,
        top_p=0.001,
        repetition_penalty=1.05,
    )

    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    print("추론결과=\n")
    print(output_text)    
    
    # 새 .txt 파일을 생성하고 내용을 씁니다.
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(output_text[0])
        f.close()

    print(f"'{txt_path}' 파일이 생성되었습니다.")   

  0%|          | 0/62 [00:00<?, ?it/s]

처리중 txt_path= ./data/김광무_146_ocr.txt


  2%|▏         | 1/62 [00:10<11:05, 10.92s/it]

추론결과=

['\n\n이 이미지에는 다음과 같은 정보가 포함되어 있습니다:\n\n- 제품 이름: SPAM\n- 제품 번호: 20080405015-227\n- 포장재질: 폴리에틸렌\n- 보관 방법: 0~10℃ 냉장보관\n- 총 내용량: 605g\n- 100g 당 칼로리: 285kcal\n- 나트륨: 930mg (47%)\n- 탄수화물: 8g (2%)\n- 당류: 1g (미만)\n- 지방: 23g (43%)\n- 트랜스지방: 0g\n- 포화지방: 7g (47%)\n- 콜레스테롤: 50mg (17%)\n- 단백질: 12g (22%)\n\n또한, 이 제품은 소비기한이 있으며, 별도의 기일까지 소비가 가능한 것으로 보입니다.']
'./data/김광무_146_ocr.txt' 파일이 생성되었습니다.
처리중 txt_path= ./data/김광무_147_ocr.txt


  3%|▎         | 2/62 [00:29<15:22, 15.38s/it]

...

추론결과=

['\n\n이 이미지에는 다음과 같은 정보가 포함되어 있습니다:\n\n1. **제품명**: 콘마요소스\n2. **식품 유형**: 소스\n3. **내용량**: 40g\n4. **영양 정보**:\n   - 나트륨: 115mg (11%)\n   - 탄수화물: 8g (2%)\n   - 지방: 6g (6%)\n   - 트랜스지방: 0g (0%)\n   - 오메가-3 지방: 1.9g (13%)\n   - 당류: 6g (6%)\n   - 조지방: 12g (22%)\n   - 브레스테롤: 5mg (1%)\n   - 칼슘: 0.4g (1%)\n   - 비타민 A: 11% (기준치에 대한 비율)\n   - 비타민 C: 11% (기준치에 대한 비율)\n   - 비타민 E: 11% (기준치에 대한 비율)\n   - 칼륨: 11% (기준치에 대한 비율)\n   - 철분: 11% (기준치에 대한 비율)\n   - 단백질: 11% (기준치에 대한 비율)\n   - 탄수화물: 11% (기준치에 대한 비율)\n   - 지방: 11% (기준치에 대한 비율)\n   - 칼슘: 11% (기준치에 대한 비율)\n   - 비타민 A: 11% (기준치에 대한 비율)\n   - 비타민 C: 11% (기준치에 대한 비율)\n   - 비타민 E: 11% (기준치에 대한 비율)\n   - 칼륨: 11% (기준치에 대한 비율)\n   - 철분: 11% (기준치에 대한 비율)\n   - 단백질: 11% (기준치에 대한 비율)\n   - 탄수화물: 11% (기준치에 대한 비율)\n   - 지방: 11% (기준치에 대한 비율)\n   - 칼슘: 11% (기준치에 대한 비율)\n   - 비타민 A: 11% (기준치에 대한 비율)\n   - 비타민 C: 11% (기준치에 대한 비율)\n   - 비타민 E: 11% (기준치에 대한 비율)\n   - 칼륨: 11% (기준치에 대한 비율)\n   - 철분: 11% (기준치에 대한 비율)\n   - 단백질: 11% (기준치에 대한 비율)\n   - 탄수화물: 11

 98%|█████████▊| 61/62 [18:47<00:24, 24.37s/it]

추론결과=

['\n\n이 이미지에는 다음과 같은 정보가 포함되어 있습니다:\n\n- 제품명: 백설 돼지갈비양념\n- 식품의 유형: 소스 (살균제품)\n- 내용량: 500g\n- 제조사: 씨제이제일제당(주) 충청남도 논산시 연무읍 축본2길 16\n- 포장재질: 용기 - 유리, 뚜껑 - 폴리염화비닐\n- 소비기한: 하단 표기일까지\n- 보관방법: 실온 보관, 개봉 후 냉장보관 권장\n- 품목보고번호: 20020464088-38\n- 원재료명: 혼합간장{아미노산액(탈지대두: 외국산), 양조간장원액(탈지대두: 외국산), 정제소금, 기타과당}, 설탕, 기타과당, 배류{배(국산), 비타민C}, 정제수, 양파퓨레(중국산), 사과퓨레{사과(국산), 비타민C}, 다진마늘, 다진생강, 카라멜색소Ⅲ, 후춧가루, 양미증진제, 정제소금, 산도조절제, L-글루탐산나트륨(향미증 진제), 잔탄검, 청양고추액기스C, 감귤류추출물\n- 대두, 밀 함유\n- 반품 및 교환장소: 전국판매점\n- 유통기한: 2026.02.18 20:18 DBS']
'./data/차지예_009_ocr.txt' 파일이 생성되었습니다.
처리중 txt_path= ./data/차지예_008_ocr.txt


100%|██████████| 62/62 [19:00<00:00, 18.40s/it]

추론결과=

['\n\n이 이미지에는 다음과 같은 정보가 포함되어 있습니다:\n\n- 제품 유형: 과자(유처리제품)\n- 품목보고번호: 19960242067421\n- 내포장재질: 플리프로필렌\n- 제조판매원: 롯데웰푸드(주) 대전광역시 대덕구 문평동로 18번길 21\n- 원재료명: 밀가루(밀;미국산), 혼합식용유[팜유(말레이시아산),야자유(외국산;필리핀,인도네시아,말레이시아 등), 토코페롤(혼합형)], 쇼트닝[가공유지(팜핵에스테르화유;말레이시아산),우지(호주산),팜유,팜스테라린유,레시틴],설탕, 기타과당,산도조절제3종, 쇼트닝Ⅱ, 맥아분말, 천일염, 안데스 소금(아르헨티나산), 체다치즈분말, 레시틴, 항료, 효소제2종\n- 영양정보:\n  - 총 내용량: 300g (50g x 6봉)\n  - 1봉(50g)당: 265kcal\n  - 나트륨: 520mg (26%)\n  - 탄수화물: 33g (10%)\n  - 당류: 5g (5%)\n  - 지방: 13g (24%)\n  - 트랜스지방: 0.5g 미만\n  - 포화지방: 7g (47%)\n  - 콜레스테롤: 5mg 미만\n  - 단백질: 3.4g (6%)\n\n이 정보는 제품의 구성 및 영양 정보를 제공합니다.']
'./data/차지예_008_ocr.txt' 파일이 생성되었습니다.
CPU times: user 26min 8s, sys: 4min 21s, total: 30min 29s
Wall time: 19min
